In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

device = "cuda" # 加载模型的设备

model = AutoModelForCausalLM.from_pretrained(
   "M:\model\Qwen1.5_0.5b_chat",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("M:\model\Qwen1.5_0.5b_chat")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
llm = HuggingFacePipeline(pipeline=pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from langchain_community.document_loaders import PDFPlumberLoader
file_path = '1关于数字化改革总体方案的汇报（省委改革办）.pdf'
loader = PDFPlumberLoader(file_path)
data = loader.load()


In [3]:
# 文本分割
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
all_splits = text_splitter.split_documents(data)

In [4]:
# 向量化存储
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "./bge-large-zh-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
bgeEmbeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

No sentence-transformers model found with name ./bge-large-zh-v1.5. Creating a new one with MEAN pooling.
c:\Users\gu\AppData\Local\conda\conda\envs\py310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
# 向量数据库
from langchain.vectorstores import FAISS
vector = FAISS.from_documents(all_splits, bgeEmbeddings)


In [6]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# retriever.invoke("省委改革办数字化总体方案要求有哪些？")

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import BaseChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""仅根据所提供的上下文回答以下问题:

<context>
{context}
</context>

问题: {question}""")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
retriever_chain: RunnableSequence = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    # | StrOutputParser()
)

In [8]:
ans1 = retriever_chain.invoke("省委改革办数字化总体方案要求有哪些？")
# ans1 = retriever_chain.invoke("省委改革办数字化总体方案要求有哪些？")
print(ans1)


Human: 仅根据所提供的上下文回答以下问题:

<context>
[Document(page_content='全 省 数 字 化 改 革 大 会\n会\n议\n资\n料\n（一）\n中共浙江省委全面深化改革委员会办公室\n年 月\n２０２１ ２\n书书书', metadata={'source': '1关于数字化改革总体方案的汇报（省委改革办）.pdf', 'file_path': '1关于数字化改革总体方案的汇报（省委改革办）.pdf', 'page': 0, 'total_pages': 44, 'CreationDate': "D:20210217220844+08'00'", 'Author': 'Founder Electronics Ltd', 'Creator': 'Founder PDF Library', 'Producer': 'Founder PDF Library', 'ModDate': "D:20210217213532+08'00'", 'start_index': 0}), Document(page_content='关于数字化改革总体方案的汇报\n省委改革办\n根据会议安排，现就数字化改革总体方案（以下简称《总体方\n案》）有关情况汇报如下。\n一、起草背景\n省委十四届八次全会作出了以数字化改革为总抓手，撬动各\n领域各方面改革的决策部署。袁家军书记亲自谋划、亲自推动，在\n省委党校党的十九届五中全会精神集中轮训班上提出了“七个关\n键”的顶层设计，此后又分别于 年 月 日、 年 月\n２０２０ １２ ２９ ２０２１ １ ４\n日、 月 日、 月 日、 月 日、 月 日主持召开 次专题会\n１ ２２ ２ ７ ２ ９ ２ １４ ６\n议，听取汇报、作出指示，逐一研究明确数字化改革的定义内涵、目\n标任务、体系架构、方法路径、推进机制等重大问题，为数字化改革\n搭建了“四梁八柱”。郑栅洁省长多次强调，要充分发挥政府数字\n化转型先发优势，推进数字化改革，提高政府治理现代化水平。陈\n金彪秘书长、王昌荣书记、高兴夫副省长等省领导指导推动总体方\n案和相关系统方案起草工作。省委改革办、省大数据局切实履行\n牵头职责，会同省委办公厅、省政府办公厅、省委政法委、省发展改\n革委、省经信厅等单位认真研究，综合集成支撑平台方案和党政机

In [9]:
ans2 = retriever_chain.invoke("省委改革办数字化改革总体方案的汇报中的“1 5 2”任务分别指哪些？")
print(ans2)

Human: 仅根据所提供的上下文回答以下问题:

<context>
[Document(page_content='全 省 数 字 化 改 革 大 会\n会\n议\n资\n料\n（一）\n中共浙江省委全面深化改革委员会办公室\n年 月\n２０２１ ２\n书书书', metadata={'source': '1关于数字化改革总体方案的汇报（省委改革办）.pdf', 'file_path': '1关于数字化改革总体方案的汇报（省委改革办）.pdf', 'page': 0, 'total_pages': 44, 'CreationDate': "D:20210217220844+08'00'", 'Author': 'Founder Electronics Ltd', 'Creator': 'Founder PDF Library', 'Producer': 'Founder PDF Library', 'ModDate': "D:20210217213532+08'00'", 'start_index': 0}), Document(page_content='案，多次修改，不断迭代，形成了今天《总体方案》汇报稿。\n二、主要内容\n《总体方案》分为总体要求、重点任务、保障体系 个板块。\n３\n（一）总体要求。包括指导思想、方法路径、主要目标 块内\n３\n容。\n关于指导思想：突出忠实践行“八八战略”、奋力打造“重要窗\n口”的主题主线，以数字化改革撬动各领域各方面改革，强调运用\n数字化技术、数字化思维、数字化认知对省域治理的体制机制、组\n织架构、方式流程、手段工具进行全方位系统性重塑，聚焦“七个\n关键”，围绕一体化、全方位、制度重塑，以重大任务和年度目标为\n切入点，推进多部门多场景系统集成应用，推动各地各部门流程再\n造、数字赋能、高效协同、整体智治，实现质量变革、效率变革、动力\n变革，高水平推进省域治理现代化，争创社会主义现代化先行省。\n关于方法路径：用好 字开发方法，将“业务协同模型”和“数\nＶ\n据共享模型”贯穿到数字化改革的各领域、各方面、全过程。 字\nＶ\n下行阶段，全面梳理党政机关核心业务，从治理与服务两个维度赋\n予定义，从宏观到微观，实现核心业务数字化。 字上行阶段，再\nＶ\n造业务流程，将核心业务组装集成为“一件事”，推进原